## 09 - How to append data to existing datacube stored in team S3 storage
### A DeepESDL example notebook 

This notebook demonstrates how to append new data to an existing datacube. This cannot be done using xcube directly yet. 

Please, also refer to the [DeepESDL documentation](https://deepesdl.readthedocs.io/en/latest/guide/jupyterlab/) and visit the platform's [website](https://www.earthsystemdatalab.net/) for further information!

Brockmann Consult, 2023

-----------------

**This notebook runs with the python environment `deepesdl-xcube-1.1.2`, please checkout the documentation for [help on changing the environment](https://deepesdl.readthedocs.io/en/latest/guide/jupyterlab/#python-environment-selection-of-the-jupyter-kerne).**

First, lets create a small cube, which we can later on append data to. We will use ESA CCI data for this. Please head over to 3 - Generate CCI data cubes to get more details about the xcube-cci data store :)

In [1]:
from xcube.core.store import new_data_store
import os

In [2]:
store = new_data_store('cciodp')
store

Next, we create a cube containing only 2 days of data:

In [3]:
dataset = store.open_data('esacci.SST.day.L4.SSTdepth.multi-sensor.multi-platform.OSTIA.2-1.sst', 
                          variable_names=['analysed_sst'],
                          time_range=['1981-09-01','1981-09-02'])

dataset

<xarray.Dataset>
Dimensions:       (time: 2, lat: 3600, lon: 7200, bnds: 2)
Coordinates:
  * lat           (lat) float32 -89.97 -89.93 -89.88 ... 89.88 89.93 89.97
    lat_bnds      (lat, bnds) float32 dask.array<chunksize=(3600, 2), meta=np.ndarray>
  * lon           (lon) float32 -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
    lon_bnds      (lon, bnds) float32 dask.array<chunksize=(7200, 2), meta=np.ndarray>
  * time          (time) datetime64[ns] 1981-09-01T12:00:00 1981-09-02T12:00:00
    time_bnds     (time, bnds) datetime64[ns] dask.array<chunksize=(2, 2), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    analysed_sst  (time, lat, lon) float32 dask.array<chunksize=(1, 1200, 2400), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7
    title:                   esacci.SST.day.L4.SSTdepth.multi-sensor.multi-pl...
    date_created:            2023-07-12T10:07:18.106673
    processing_level:        L4
    time_coverage_start:     1981-09-01T00:00:00
    time_coverage_end:       1981-09-03T00:00:00
    time_coverage_duration:  P2DT0H0M0S
    history:                 [{'program': 'xcube_cci.chunkstore.CciChunkStore...

In [4]:
dataset

<xarray.Dataset>
Dimensions:       (time: 2, lat: 3600, lon: 7200, bnds: 2)
Coordinates:
  * lat           (lat) float32 -89.97 -89.93 -89.88 ... 89.88 89.93 89.97
    lat_bnds      (lat, bnds) float32 dask.array<chunksize=(3600, 2), meta=np.ndarray>
  * lon           (lon) float32 -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
    lon_bnds      (lon, bnds) float32 dask.array<chunksize=(7200, 2), meta=np.ndarray>
  * time          (time) datetime64[ns] 1981-09-01T12:00:00 1981-09-02T12:00:00
    time_bnds     (time, bnds) datetime64[ns] dask.array<chunksize=(2, 2), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    analysed_sst  (time, lat, lon) float32 dask.array<chunksize=(1, 1200, 2400), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7
    title:                   esacci.SST.day.L4.SSTdepth.multi-sensor.multi-pl...
    date_created:            2023-07-12T10:07:18.106673
    processing_level:        L4
    time_coverage_start:     1981-09-01T00:00:00
    time_coverage_end:       1981-09-03T00:00:00
    time_coverage_duration:  P2DT0H0M0S
    history:                 [{'program': 'xcube_cci.chunkstore.CciChunkStore...

Next, save it to the team s3 storage:

To store the cube in your teams user space, please first retrieve the details from your environment variables as the following:

In [5]:
S3_USER_STORAGE_KEY = os.environ["S3_USER_STORAGE_KEY"]
S3_USER_STORAGE_SECRET = os.environ["S3_USER_STORAGE_SECRET"]
S3_USER_STORAGE_BUCKET = os.environ["S3_USER_STORAGE_BUCKET"]

You need to instantiate a s3 datastore pointing to the team bucket:

In [6]:
from xcube.core.store import new_data_store
team_store = new_data_store("s3", 
                       root=S3_USER_STORAGE_BUCKET, 
                       storage_options=dict(anon=False, 
                                            key=S3_USER_STORAGE_KEY, 
                                            secret=S3_USER_STORAGE_SECRET))


If you have stored no data to your user space, the returned list will be empty:

In [7]:
list(team_store.get_data_ids())

[]

Appending data currently only works with .zarr format and is not supported in .levels yet. 

In [8]:
output_id = 'analysed_sst.zarr'

In [9]:
team_store.write_data(dataset, output_id)

'analysed_sst.zarr'

If you list the content of you datastore again, you will now see the newly written dataset in the list:

In [10]:
list(team_store.get_data_ids())

['analysed_sst.zarr']

Now, to append new time stamps, xcube cannot be used but there is a workaround :) 

In [11]:
# needed for appending data to an existing cube saved in s3 storage
import s3fs

Connect to your team storage in S3 

In [12]:
# Connect to AWS S3 storage
fs = s3fs.S3FileSystem(anon=False, 
                       key=S3_USER_STORAGE_KEY,
                       secret=S3_USER_STORAGE_SECRET)

In [13]:
s3_client_kwargs = {'endpoint_url': "https://s3.eu-central-1.amazonaws.com"}
target_bucket_path = f"s3://{S3_USER_STORAGE_BUCKET}"

We create a new dataset, with different time stamps, which we want to append to the existing cube: 

In [14]:
dataset = store.open_data('esacci.SST.day.L4.SSTdepth.multi-sensor.multi-platform.OSTIA.2-1.sst', 
                          variable_names=['analysed_sst'],
                          time_range=['1981-09-03','1981-09-04'])

dataset

<xarray.Dataset>
Dimensions:       (time: 2, lat: 3600, lon: 7200, bnds: 2)
Coordinates:
  * lat           (lat) float32 -89.97 -89.93 -89.88 ... 89.88 89.93 89.97
    lat_bnds      (lat, bnds) float32 dask.array<chunksize=(3600, 2), meta=np.ndarray>
  * lon           (lon) float32 -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
    lon_bnds      (lon, bnds) float32 dask.array<chunksize=(7200, 2), meta=np.ndarray>
  * time          (time) datetime64[ns] 1981-09-03T12:00:00 1981-09-04T12:00:00
    time_bnds     (time, bnds) datetime64[ns] dask.array<chunksize=(2, 2), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    analysed_sst  (time, lat, lon) float32 dask.array<chunksize=(1, 1200, 2400), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7
    title:                   esacci.SST.day.L4.SSTdepth.multi-sensor.multi-pl...
    date_created:            2023-07-12T10:07:47.326729
    processing_level:        L4
    time_coverage_start:     1981-09-03T00:00:00
    time_coverage_end:       1981-09-05T00:00:00
    time_coverage_duration:  P2DT0H0M0S
    history:                 [{'program': 'xcube_cci.chunkstore.CciChunkStore...

In [15]:
# we need to create a mapper pointing to the existing cube, stored in the team s3 storage
mapper = fs.get_mapper(f'{target_bucket_path}/{output_id}')


Now we can append the new dataset to the existing datacube:

In [16]:
dataset.to_zarr(mapper, mode='a', append_dim='time', consolidated=True)    


In [17]:
list(team_store.get_data_ids())

['analysed_sst.zarr']

Check if the cube now contains the expected time stamps: 

In [18]:
ds = team_store.open_data(output_id)

As expected, we now find all four days in the datacube. **Please note: you are responsible for passing the time stamps in the right order. If you do not, this might cause trouble later on and you will need to reorder the time dimension.**

In [19]:
ds

<xarray.Dataset>
Dimensions:       (time: 4, lat: 3600, lon: 7200, bnds: 2)
Coordinates:
  * lat           (lat) float32 -89.97 -89.93 -89.88 ... 89.88 89.93 89.97
    lat_bnds      (lat, bnds) float32 dask.array<chunksize=(3600, 2), meta=np.ndarray>
  * lon           (lon) float32 -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
    lon_bnds      (lon, bnds) float32 dask.array<chunksize=(7200, 2), meta=np.ndarray>
  * time          (time) datetime64[ns] 1981-09-01T12:00:00 ... 1981-09-04T12...
    time_bnds     (time, bnds) datetime64[ns] dask.array<chunksize=(2, 2), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    analysed_sst  (time, lat, lon) float32 dask.array<chunksize=(1, 1200, 2400), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7
    date_created:            2023-07-12T10:07:47.326729
    history:                 [{'cube_params': {'time_range': ['1981-09-03T00:...
    processing_level:        L4
    time_coverage_duration:  P2DT0H0M0S
    time_coverage_end:       1981-09-05T00:00:00
    time_coverage_start:     1981-09-03T00:00:00
    title:                   esacci.SST.day.L4.SSTdepth.multi-sensor.multi-pl...

It is also possible to append a variable with the same dimensions to an existing datacube:

In [20]:
dataset = store.open_data('esacci.SST.day.L4.SSTdepth.multi-sensor.multi-platform.OSTIA.2-1.sst', 
                          variable_names=['sea_ice_fraction'],
                          time_range=['1981-09-01','1981-09-04'])

dataset

<xarray.Dataset>
Dimensions:           (lat: 3600, bnds: 2, lon: 7200, time: 4)
Coordinates:
  * lat               (lat) float32 -89.97 -89.93 -89.88 ... 89.88 89.93 89.97
    lat_bnds          (lat, bnds) float32 dask.array<chunksize=(3600, 2), meta=np.ndarray>
  * lon               (lon) float32 -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
    lon_bnds          (lon, bnds) float32 dask.array<chunksize=(7200, 2), meta=np.ndarray>
  * time              (time) datetime64[ns] 1981-09-01T12:00:00 ... 1981-09-0...
    time_bnds         (time, bnds) datetime64[ns] dask.array<chunksize=(4, 2), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    sea_ice_fraction  (time, lat, lon) float32 dask.array<chunksize=(1, 1800, 1800), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7
    title:                   esacci.SST.day.L4.SSTdepth.multi-sensor.multi-pl...
    date_created:            2023-07-12T10:08:12.332909
    processing_level:        L4
    time_coverage_start:     1981-09-01T00:00:00
    time_coverage_end:       1981-09-05T00:00:00
    time_coverage_duration:  P4DT0H0M0S
    history:                 [{'program': 'xcube_cci.chunkstore.CciChunkStore...

Now we can append the new dataset with the additional variable to the existing datacube:

In [21]:
dataset.to_zarr(mapper, mode='a', consolidated=True)    


Check if the cube now contains the expected new variable: 

In [22]:
ds = team_store.open_data(output_id)

In [23]:
ds

<xarray.Dataset>
Dimensions:           (time: 4, lat: 3600, lon: 7200, bnds: 2)
Coordinates:
  * lat               (lat) float32 -89.97 -89.93 -89.88 ... 89.88 89.93 89.97
    lat_bnds          (lat, bnds) float32 dask.array<chunksize=(3600, 2), meta=np.ndarray>
  * lon               (lon) float32 -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
    lon_bnds          (lon, bnds) float32 dask.array<chunksize=(7200, 2), meta=np.ndarray>
  * time              (time) datetime64[ns] 1981-09-01T12:00:00 ... 1981-09-0...
    time_bnds         (time, bnds) datetime64[ns] dask.array<chunksize=(2, 2), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    analysed_sst      (time, lat, lon) float32 dask.array<chunksize=(1, 1200, 2400), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) float32 dask.array<chunksize=(1, 1800, 1800), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7
    date_created:            2023-07-12T10:08:12.332909
    history:                 [{'cube_params': {'time_range': ['1981-09-01T00:...
    processing_level:        L4
    time_coverage_duration:  P4DT0H0M0S
    time_coverage_end:       1981-09-05T00:00:00
    time_coverage_start:     1981-09-01T00:00:00
    title:                   esacci.SST.day.L4.SSTdepth.multi-sensor.multi-pl...

**In the above example you see, that the chunking is different for the new variable. You can ensure the desired chunkind before writing the data to a cube by specifying it in the encoding. To learn about chunking, head over to the example notebook 10 - Chunking of Datasets**

Alright, now you know how to append new time stamps or variables to an existing cube - let's clean up our example :) 

In [24]:
team_store.delete_data(output_id)

In [25]:
list(team_store.get_data_ids())

[]